In [1]:
from datetime import datetime, timezone

from dateutil.relativedelta import relativedelta

from github_analysis.data_frame_processing import (
    group_pull_requests_by_repositories,
    pull_request_reviews_as_data_frame,
    pull_requests_as_data_frame,
)
from github_analysis.github.client.users import PullRequestStates
from github_analysis.github.service import GitHubService

PAGINATION_STEP_AMOUNT = 100
FILTER_STATES: list[PullRequestStates] = ["MERGED"]
NOW = datetime.now().replace(tzinfo=timezone.utc)
ONE_MONTH_AGO = NOW - relativedelta(months=1)
AGGREGATION_TIME = ONE_MONTH_AGO
USERNAME = "kentcdodds"

github_service = GitHubService()

(
    pull_request_reviews,
    merged_pull_requests,
) = await github_service.get_users_reviews_and_pull_requests(
    username=USERNAME,
    pagination_step_amount=PAGINATION_STEP_AMOUNT,
    from_date=AGGREGATION_TIME,
    pull_request_states=FILTER_STATES,
)

In [2]:
merged_pull_requests_data_frame = pull_requests_as_data_frame(
    pull_requests=merged_pull_requests
)
merged_pull_requests_data_frame

created_at,pull_request_number,amount_of_comments,repository_name
"datetime[μs, UTC]",str,i64,str
2024-09-30 16:54:39 UTC,"""#321""",0,"""kentcdodds/bookshelf"""
2024-09-30 16:53:52 UTC,"""#102""",0,"""kentcdodds/testing-react-apps"""
2024-09-24 12:50:42 UTC,"""#16""",0,"""vitejs/companies-using-vite"""
2024-09-21 14:56:44 UTC,"""#1735""",1,"""skillrecordings/products"""
2024-09-19 18:59:22 UTC,"""#1729""",1,"""skillrecordings/products"""
2024-09-10 17:15:01 UTC,"""#1715""",1,"""skillrecordings/products"""
2024-09-09 22:48:19 UTC,"""#217""",0,"""epicweb-dev/epicshop"""


In [3]:
grouped_merged_pull_requests = group_pull_requests_by_repositories(
    pull_requests=merged_pull_requests_data_frame
)
grouped_merged_pull_requests

repository_name,amount_of_pull_requests
str,u32
"""skillrecordings/products""",3
"""kentcdodds/bookshelf""",1
"""vitejs/companies-using-vite""",1
"""epicweb-dev/epicshop""",1
"""kentcdodds/testing-react-apps""",1


In [4]:
pull_request_reviews_as_data_frame(pull_request_reviews)

shape: (0, 0)
┌┐
╞╡
└┘